<a href="https://colab.research.google.com/github/martings/My_own_single_cell_toolkit/blob/main/Integracion_de_datasets_en_Seurat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('Seurat')
#install.packages('tidyverse')
library(Seurat)
library(tifyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘plyr’, ‘tensor’, ‘BH’, ‘sitmo’, ‘globals’, ‘listenv’, ‘parallelly’, ‘zoo’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’, ‘RcppTOML’, ‘here’, ‘gplots’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘sp’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘spatstat.data’, ‘spatstat.random’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘goftest’, ‘abind’, ‘deldir’, ‘polyclip’, ‘FNN’, ‘dqrng’, ‘cowplot’, ‘fitdistrplus’, ‘future’, ‘future.apply’, ‘ggrepel’, ‘ggridges’, ‘ica’, ‘igraph’, ‘irlba’, ‘leiden’, ‘lmtest’, ‘matrixStats’, ‘miniUI’, ‘patchwork’, ‘pbapply’, ‘plotly’, ‘png’, ‘progressr’, ‘RANN’, ‘Rcpp’, ‘RcppAnnoy’, ‘reticulate’, ‘ROCR’, ‘Rtsne’, ‘scattermore’, ‘sctransform’, ‘SeuratObject’, ‘shiny’, ‘spatstat.explore’, ‘spatstat.geom’, ‘uwot’, ‘RcppEigen’, ‘RcppProgress’




In [ ]:
install.packages("gt", quiet=TRUE)
install.packages("tidyverse", quiet=TRUE)
suppressMessages({
  library(gt)
  library(tidyverse)
})

iris %>%
  head() %>%
  gt::gt() %>%
  gt::tab_spanner_delim(delim = ".") %>%
  gt::cols_move_to_start(columns = Species) %>%
  gt::fmt_number(
    columns = c(Sepal.Length, Sepal.Width, Petal.Length, Petal.Width),
    decimals = 1
  ) %>%
  gt::tab_header(
    title = "The iris dataset",
    subtitle = "All about iris setosa, versicolor, and virginica"
  ) %>%
  gt::tab_source_note(
    source_note = "The data were collected by Anderson (1935)."
  ) %>%
  gt::opt_row_striping()

In [ ]:
print(R.version.string)

In [ ]:
suppressMessages({
    library(tidyverse)
})

In [1]:
#Cargo los dataset de entrada para realizar la integracion

Khaled_total<-readRDS("/SC_MG_Data/seurat_Khaled_geo_data.rds")
AdultMG_Spike<-readRDS("/SC_MG_Data/AdultMG_combined_BackUp.rds")
AdultMG_Visvader<-readRDS("/SC_MG_Data/Visvader.10x.combined.BackUP.dim30.rds")

#Armo una lista con todos los objetos a integrar
seurat_object_list<-c("Khaled_total","AdultMG_Spike","AdultMG_Visvader")
pbmc.list <- list()
for (i in seurat_object_list) {
  pbmc.list[i] <- list(get(i))
}
#Borro los archivos originales de memoria RAM
rm(Khaled_total)
rm(AdultMG_Spike)
rm(AdultMG_Visvader)

#Transformo y normalizo los datos ori
for (i in names(pbmc.list)) {
  pbmc.list[[i]] <- SCTransform(pbmc.list[[i]], verbose = T)
}


pbmc.features <- SelectIntegrationFeatures(object.list = pbmc.list, nfeatures = 3000)
pbmc.list <- PrepSCTIntegration(object.list = pbmc.list, anchor.features = pbmc.features)

# This command returns dataset 5.  We can also specify multiple refs. (i.e. c(5,6))
reference_dataset <- which(names(pbmc.list) == "Khaled_total")

pbmc.anchors <- FindIntegrationAnchors(object.list = pbmc.list, normalization.method = "SCT", 
                                       anchor.features = pbmc.features, reference = reference_dataset)
rm(pbmc.list)
pbmc.integrated <- IntegrateData(anchorset = pbmc.anchors, normalization.method = "SCT")
saveRDS(pbmc.integrated, "pbmc.integrated.backup.rds")
rm(pbmc.anchors)
pbmc.integrated <- RunPCA(object = pbmc.integrated, verbose = T)
pbmc.integrated <- RunUMAP(object = pbmc.integrated, dims = 1:30)

saveRDS(pbmc.integrated, "pbmc.integrated.post.rds")

ERROR: ignored